Command line interface
====================

xclim provides the  `xclim` command line executable to perform basic indicator
computation easily without having to start up a full python environment. However, not
all indicators listed in [Climate Indicators](../indicators.rst) are available through this tool.

Its use is simple. Type the following to see the usage message:

In [ ]:
!xclim --help

To list all available indicators, use the "indices" subcommand:

In [ ]:
!xclim indices

For more information about a specific indicator, you can either use the `info` subcommand or directly access the `--help` message of the indicator. The former gives more information about the metadata while the latter only prints the usage. Note that the module name (`atmos`, `land` or `seaIce`) is mandatory.

In [ ]:
!xclim info liquidprcptot

In the usage message, `VAR_NAME` indicates that the passed argument must match a variable in the input dataset.

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import warnings
warnings.filterwarnings('ignore', 'implicitly registered datetime converter')
%matplotlib inline
xr.set_options(display_style='html')

time = pd.date_range('2000-01-01', periods=366)
tasmin = xr.DataArray(-5 * np.cos(2 * np.pi * time.dayofyear / 365) + 273.15, dims=("time"), 
                      coords={'time': time}, attrs={'units':'K'})
tasmax = xr.DataArray(-5 * np.cos(2 * np.pi * time.dayofyear / 365) + 283.15, dims=("time"),
                      coords={'time': time}, attrs={'units':'K'})
pr = xr.DataArray(np.clip(10 * np.sin(18 * np.pi * time.dayofyear / 365), 0, None), dims=("time"),
                      coords={'time': time}, attrs={'units':'mm/d'})
ds = xr.Dataset({'tasmin': tasmin, 'tasmax': tasmax, 'pr': pr})
ds.to_netcdf('example_data.nc')

Computing indicators
-------------------------------

So let's say we have the following toy dataset:

In [ ]:
import xarray as xr
ds = xr.open_dataset('example_data.nc')
ds

In [ ]:
import matplotlib.pyplot as plt
fig, (axT, axpr) = plt.subplots(1, 2, figsize=(10, 5))
ds.tasmin.plot(label='tasmin', ax=axT)
ds.tasmax.plot(label='tasmax', ax=axT)
ds.pr.plot(ax=axpr)
axT.legend()

To compute an indicator, say the monthly solid precipitation accumulation, we simply call:

In [ ]:
!xclim -i example_data.nc -o out1.nc solidprcptot --pr pr --tas tasmin --freq MS

In this example, we decided to use `tasmin` for the `tas` variable. We didn't need to provide the `--pr` parameter as our data has the same name. 

Finally, more than one indicators can be computed to the output dataset by simply chaining the calls:

In [ ]:
!xclim -i example_data.nc -o out2.nc liquidprcptot --tas tasmin --freq MS tropical_nights --thresh "2 degC" --freq MS

Let's see the outputs:

In [ ]:
ds1 = xr.open_dataset('out1.nc')
ds2 = xr.open_dataset('out2.nc', decode_timedelta=False)

fig, (axPr, axTn) = plt.subplots(1, 2, figsize=(10, 5))
ds1.solidprcptot.plot(ax=axPr, label=ds1.solidprcptot.long_name)
ds2.liquidprcptot.plot(ax=axPr, label=ds2.liquidprcptot.long_name)
ds2.tropical_nights.plot(ax=axTn, marker='o')
axPr.legend()

In [ ]:
ds1.close()

In [ ]:
ds2.close()